In [52]:
import mne 
from glob import glob 
import os
import matplotlib
from openpyxl import load_workbook
import numpy as np

In [53]:
matplotlib.use('Qt5Agg') 

In [54]:
#quanti soggetti ho nella mia cartella?
subs_file_path=glob("..\..\DATASET PRINCIPALE\CartellaEEG/*.vhdr")
print(len(subs_file_path))

2


In [55]:
#creo vettore file excel---- !ATTENZIONE FAR RUNNARE IL FILE EXCEL.YPTR per aggiustare gli excel
excel_file_path=glob("..\..\DATASET PRINCIPALE\Cartellaexcel/*.xlsx")
print(len(excel_file_path))

2


In [176]:
#Creo la funzione che legge e processa i dati 
def prep_data(file_path):
    data= mne.io.read_raw_brainvision(file_path, preload=True)
    data.set_eeg_reference();
    data.notch_filter(np.arange(50,100,50), filter_length='auto',phase='zero')
    data.notch_filter(np.arange(20,40,20), filter_length='auto',phase='zero')
    data.notch_filter(np.arange(10,20,10), filter_length='auto',phase='zero')
    data.filter(l_freq=0.1,h_freq=100)
    data.resample(250)
    return data

#Creo la funzione che estrae la matrice delle annotazioni dai file excel 
def get_vectors(data_path):

    book=load_workbook(data_path)
    sheet=book.active
    
    #onset
    onset=[]
    for row in sheet['a2':'a22249']:
        for cel in row:
               onset.append(cel.value)
                
    onset=np.array(onset)
    onset=onset*0.001
    
    #duration 
    duration=np.ones((22248,), dtype=int)*0.01
    
    #description 
    animal=['animal']
    description=[]
    for row in sheet ['e2':'e22249']:
        for cel in row:
            if(cel.value in animal):
                description.append(cel.value)
            else: 
                description.append('E 1')
    return [onset, duration, description] 

#creo la funzione che applica le annotazioni 
def fix_events(data_vector, matrix):
    myannot=[mne.Annotations(onset=matrix[i][0], duration=matrix[i][1],description=matrix[i][2])for i in range(len(excel_file_path))]
    for i in range(len(excel_file_path)):
        data_vector[i].set_annotations(myannot[i])
    
    return data_vector

#creo la funzione che estrae le epochs
def get_epochs(data_vector):
    tmin=-0.100 #partono 100 millisecondi prima di un evento 
    tmax=1 #finiscon 1000 milli secondo dopo 
    baseline=(None,0)
    epochs=[mne.Epochs(i,events=mne.events_from_annotations(i)[0],tmin=tmin,tmax=tmax,baseline=baseline, preload=True)for  i in data_vector]
   
    for i in range(len(data_vector)):
        #reject  bad epochs
        reject_criteria = dict(eeg=150e-6)
        flat_criteria = dict(eeg=1e-6)
        epochs[i].drop_bad(reject=reject_criteria, flat=flat_criteria)
        
    return epochs 

In [57]:
#funzione preprocessing
prep_data_sub_vector= [prep_data(i)for i in subs_file_path]

Extracting parameters from ..\..\DATASET PRINCIPALE\CartellaEEG\sub-01_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3035739  =      0.000 ...  3035.739 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 6601 samples (6.601 sec)

Setting up band-stop filter from 19 - 21 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firw

In [58]:
#estraggo la matrice delle annotazioni 
matrix=[]
for i in range(len(excel_file_path)):
    matrix=[get_vectors(i)for i in excel_file_path]

In [151]:
#sistemo gli eventi 
data_vector_ok= fix_events(prep_data_sub_vector, matrix)

In [158]:
#definisco gli eventi 
#events_vector=[mne.events_from_annotations(i)for i in data_vector_ok]

Used Annotations descriptions: ['E 1', 'animal']
Used Annotations descriptions: ['E 1', 'animal']


In [60]:
#ICA
#applico la ica per visualizzare i picchi di potenza da rimuovere
ica = mne.preprocessing.ICA(n_components=30, max_iter=1000,random_state=0)
for i in range(len(data_vector_ok)):
    ica.fit(data_vector_ok[i].filter(0.1, 35))
    bad_idx, scores=ica.find_bads_eog(data_vector_ok[i],ch_name=['Fp1','Fp2'],threshold=1)
    ica.exclude=bad_idx
    data_vector_ok[i]=ica.apply(data_vector_ok[i], exclude=ica.exclude)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 8251 samples (33.004 sec)

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 30 components
Fitting ICA took 62.0s.
Using EOG channels: Fp1, Fp2
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- L

In [61]:
#TOLGO A MANO DEI CANALI CHE POSSONO ESSERE DISTURBANTI 
#for i in range(len(data_vector_ok)):
#    ((data_vector_ok[i].copy()).crop(tmax=10)).plot()
    
#data_vector_ok[0].plot_sensors()

In [62]:
#data_vector_ok[0].info['bads']=['Fp1', 'Fz', 'TP9', 'AF3', 'AF7', 'AFz', 'Fp2', 'F4', 'F8', 'FC2', 'FC6', 'FT10', 'F5', 'F1', 'FT7', 'FC3', 'FT8', 'F6', 'AF8', 'F2', 'FCz']
#data_vector_ok[1].info['bads']=['Fp1', 'Fz', 'TP9', 'AF3', 'AF7', 'AFz', 'Fp2', 'F4', 'F8', 'FC2', 'FC6', 'FT10', 'F5', 'F1', 'FT7', 'FC3', 'FT8', 'F6', 'AF8', 'F2', 'FCz']

In [178]:
epochs_vector=get_epochs(data_vector_ok)

Used Annotations descriptions: ['E 1', 'animal']
Not setting metadata
Not setting metadata
22248 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 22248 events and 276 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['E 1', 'animal']
Not setting metadata
Not setting metadata
22248 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 22248 events and 276 original time points ...
0 bad epochs dropped
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
   

    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejec

    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp

    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp

    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1', 'AF7']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting

...

    Rejecting  epoch based on EEG : ['TP9', 'F1', 'F5', 'TP7', 'PO3', 'POz', 'CPz', 'FT8']
    Rejecting  epoch based on EEG : ['TP9', 'F1', 'F5', 'TP7', 'PO3', 'POz', 'CPz', 'FT8']
    Rejecting  epoch based on EEG : ['TP9', 'F1', 'F5', 'TP7', 'PO3', 'CPz', 'FT8']
    Rejecting  epoch based on EEG : ['TP9', 'F1', 'F5', 'TP7', 'PO3', 'CPz', 'FT8']
    Rejecting  epoch based on EEG : ['TP9', 'F1', 'F5', 'TP7', 'PO3', 'CPz', 'FT8']
    Rejecting  epoch based on EEG : ['TP9', 'F1', 'F5', 'TP7', 'PO3', 'FT8']
    Rejecting  epoch based on EEG : ['TP9', 'F1', 'F5', 'TP7', 'PO3']
    Rejecting  epoch based on EEG : ['TP9', 'F1', 'F5', 'TP7', 'PO3']
    Rejecting  epoch based on EEG : ['TP9', 'F1', 'F5', 'TP7']
    Rejecting  epoch based on EEG : ['F1', 'F5', 'TP7']
    Rejecting  epoch based on EEG : ['F1', 'F5', 'TP7']
    Rejecting  epoch based on EEG : ['F5', 'TP7']
    Rejecting  epoch based on EEG : ['F5', 'TP7']
    Rejecting  epoch based on EEG : ['F5', 'TP7']
    Rejecting  epoch bas

    Rejecting  epoch based on EEG : ['Fp1', 'FC5', 'T7', 'T8', 'FT10', 'FC6', 'FC2', 'F1', 'F5', 'FT7', 'TP7', 'PO3', 'FC4', 'FT8', 'F6', 'AF4', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'FC5', 'T7', 'T8', 'FT10', 'FC6', 'FC2', 'F1', 'F5', 'FT7', 'TP7', 'PO3', 'POz', 'FC4', 'FT8', 'F6', 'AF4', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'FC5', 'T7', 'T8', 'FT10', 'FC6', 'FC2', 'F1', 'F5', 'FT7', 'C5', 'TP7', 'PO3', 'POz', 'CPz', 'FC4', 'FT8', 'F6', 'AF4', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'FC5', 'T7', 'T8', 'FT10', 'FC6', 'FC2', 'F1', 'F5', 'FT7', 'C5', 'TP7', 'PO3', 'POz', 'CPz', 'FC4', 'FT8', 'F6', 'AF4', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'FC5', 'T7', 'T8', 'FT10', 'FC6', 'FC2', 'F1', 'F5', 'FT7', 'C5', 'TP7', 'PO3', 'POz', 'CPz', 'FC4', 'FT8', 'F6', 'AF4', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'FC5', 'T7', 'T8', 'FT10', 'FC6', 'FC2', 'F1', 'F5', 'FT7', 'C5', 'TP7', 'PO3', 'POz', 'CPz', 'FC4', 'FT8', 'F6', 'AF4', 'F2']
    Rej

    Rejecting  epoch based on EEG : ['Fp1', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'Pz', 'T8', 'FT10', 'FC6', 'FC2', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'PO3', 'PO4', 'CPz', 'FC4', 'FT8', 'F6', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'FT9', 'FC5', 'FC1', 'T7', 'TP9', 'CP5', 'T8', 'FT10', 'FC6', 'FC2', 'AFz', 'F1', 'F5', 'FT7', 'C1', 'C5', 'TP7', 'CP3', 'PO4', 'CPz', 'FC4', 'FT8', 'F6', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'FT9', 'FC5', 'FC1', 'T7', 'T8', 'FT10', 'FC6', 'FC2', 'AFz', 'F1', 'F5', 'FT7', 'C1', 'C5', 'TP7', 'PO4', 'CPz', 'FT8', 'F6', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'FT9', 'FC5', 'FC1', 'T8', 'FT10', 'FC6', 'FC2', 'AFz', 'F1', 'F5', 'FT7', 'C1', 'C5', 'TP7', 'PO4', 'CPz', 'FT8', 'F6', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'FT9', 'FC5', 'FC1', 'FT10', 'FC6', 'FC2', 'AFz', 'F1', 'F5', 'FT7', 'C1', 'C5', 'TP7', 'PO4', 'CPz', 'FT8', 'F6']
    Rejecting  epoch based on EEG : ['Fp1', 'FT9', 'FC5',

    Rejecting  epoch based on EEG : ['FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'FT10', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'FC4', 'FT8', 'F2']
    Rejecting  epoch based on EEG : ['FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'FT10', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'FC4', 'FT8', 'F2']
    Rejecting  epoch based on EEG : ['FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'FT10', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'FC4', 'FT8', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP2', 'T8', 'FT10', 'FC6', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'CPz', 'CP4', 'C2', 'FC4', 'FT8', 'F6', 'F2']
    Rejecting  epoch based on EEG : ['Fp1', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'Pz', 'P3', 'P7', 'CP2', 'T8', 'FT10', 'FC6', 'F4', 'F8', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'PO3', 'POz', 'PO4', 'CPz', 'CP4', 'C2', 'FC4', 'FT8', 

    Rejecting  epoch based on EEG : ['Fp1', 'Fz', 'FC2', 'AF7', 'F1', 'FCz']
    Rejecting  epoch based on EEG : ['Fp1', 'Fz', 'FC2', 'AF7', 'F1', 'FCz']
    Rejecting  epoch based on EEG : ['Fp1', 'FC2', 'FCz']
    Rejecting  epoch based on EEG : ['Fp1', 'FC2', 'FCz']
    Rejecting  epoch based on EEG : ['Fp1', 'FC2', 'FCz']
    Rejecting  epoch based on EEG : ['Fp1', 'FC2', 'FCz']
    Rejecting  epoch based on EEG : ['Fp1', 'FC2', 'FCz']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based o

In [186]:
animali=epochs_vector[0]['10002'].average()

In [197]:
#att_Oz=[17,16,18],
#att_Poz=[45,46,48,49,47]
roi_dict = dict(
                att_Oz=[17,16,18],
                att_Poz=[45,46,48,49,47]
               )

roi_evoked = mne.channels.combine_channels(animali, roi_dict, method='mean')
print(roi_evoked.info['ch_names'])
roi_evoked.plot()

['att_Oz', 'att_Poz']


<Figure size 1280x600 with 1 Axes>